In [1]:
# Displays figures in line
# %matplotlib inline

# Import dependencies 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests 
import json
plt.style.use('ggplot')
import gmaps
# import gmaps.datasets
import datetime

# Google developer API key
from config import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

from pprint import pprint

In [2]:
# url = "https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Public_Safety_WebMercator/MapServer/24/query?where=1%3D1&outFields=*&outSR=4326&f=json"
url = "https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Public_Safety_WebMercator/MapServer/24/query?where=1%3D1&outFields=*&outSR=4326&f=json&returnIdsOnly=true"

In [3]:
response = requests.get(url)

resp_dict = json.loads(response.text)

objectIds = resp_dict["objectIds"]

objectId_count = len(objectIds)

# print(objectId_count)
  
crash_data = []  
count = 0
start = 0
end = 0
batch = 125

while count < objectId_count:

    end = end + batch
    url = "https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Public_Safety_WebMercator/MapServer/24/query?where=1%3D1&outFields=*&outSR=4326&f=json&objectIds= "+ str(objectIds[start:end]).replace('[','').replace(']','')
# url = 'https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Public_Safety_WebMercator/MapServer/24/query?where=1%3D1&outFields=*&outSR=4326&f=json&objectIds= ' + str(objectIds[0:125]).replace('[','').replace(']','')
 
# url = 'https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Public_Safety_WebMercator/MapServer/24/query?where=1%3D1&outFields=*&outSR=4326&f=json&objectIds=47221836,47221838,47221481'
    
    response_new = requests.get(url).json()
    start = end   
    count = count + batch 
    data = response_new['features']
    
    #pprint(start)
    #pprint(end)
#     pprint(data)


# print(len(data))
    i = 0
    for x in data:
        crash_data.append(data[i]['attributes'])
        i = i + 1
# print(crash_data)
#response_new = requests.get(url)
#print(response_new.text)

# for x in response_new['features']:
#         crash_data.append(x)

In [4]:
master_df = pd.DataFrame(crash_data)
master_df
# data = response['features']
# crash_data = []

,ADDRESS,BICYCLISTSIMPAIRED,CCN,CRIMEID,DRIVERSIMPAIRED,EVENTID,FATAL_BICYCLIST,FATAL_DRIVER,FATAL_PEDESTRIAN,FROMDATE,...,TOTAL_GOVERNMENT,TOTAL_PEDESTRIANS,TOTAL_TAXIS,TOTAL_VEHICLES,UNKNOWNINJURIES_BICYCLIST,UNKNOWNINJURIES_DRIVER,UNKNOWNINJURIES_PEDESTRIAN,WARD,XCOORD,YCOORD
0,1401 15TH STREET NW,0,18096710,27706990,0,{D73A37A4-B9CD-4657-AC5C-B33A8D505718},0,0,0,1.528934e+12,...,0,1,0,1,0,0,0,Ward 2,397029.339,137899.972
1,2319 NORTH CAPITOL STREET NE,0,18096618,27707056,0,{1EA45C56-9F77-4DF3-8B2E-B90A2336C74B},0,0,0,1.528934e+12,...,0,0,1,2,0,0,0,Ward 5,399246.408,139270.466
2,1251 BLADENSBURG ROAD NE,0,18096585,27707062,0,{24F19DD5-17BF-46F4-BFCB-813F7AB8F2BE},0,0,0,1.528934e+12,...,0,0,0,2,0,0,0,Ward 5,401866.750,137660.531
3,3155 MOUNT PLEASANT STREET NW,0,18096816,27707066,0,{41BB6F59-AAF2-4E2C-A632-0D00024D0892},0,0,0,1.528934e+12,...,1,0,0,2,0,0,0,Ward 1,396700.741,140290.602
4,5428 MACARTHUR BOULEVARD NW,0,18096858,27707106,0,{3E7A932C-DE8A-4342-841F-2FE2986FC46B},0,0,0,1.528934e+12,...,0,0,0,2,0,0,0,Ward 3,390707.856,140193.807
5,INTERSTATE 695 INTERSTATE BN,0,18096807,27707113,0,{41EE715E-7231-4D39-B501-81FF651379FD},0,0,0,1.528934e+12,...,0,0,0,3,0,0,0,Ward 6,400898.821,133799.562
6,5671 WESTERN AVENUE NW,0,18096899,27707137,0,{596C6204-CD6B-424D-B1B0-97EE06DE9018},0,0,0,1.528934e+12,...,0,0,0,2,0,0,0,Ward 3,393236.365,144259.275
7,INTERSTATE 395 INTERSTATE BN,0,18096888,27707141,0,{D22A87BA-3B1D-4552-B06C-E35275DFC8CA},0,0,0,1.528934e+12,...,0,0,0,2,0,0,0,Ward 2,398933.900,135111.310
8,906 S STREET NW,0,18096896,27707171,0,{32DA504A-2E20-40E4-ADBD-03CDB59F01D7},0,0,0,1.528934e+12,...,0,0,0,1,0,0,0,Ward 2,397892.030,138468.750
9,1906 5TH STREET NW,0,18112799,27732572,0,{2BA9D89D-DF0C-4ADF-995F-E78CC419BD6B},0,0,0,1.531181e+12,...,0,0,0,1,0,0,0,Ward 1,398385.378,138702.251


In [ ]:
# print(datetime.datetime.fromtimestamp(int("1398470400000")).strftime('%Y-%m-%d %H:%M:%S'))
# master_df["FROMDATE"]=pd.to_datetime(master_df["FROMDATE"]).apply(lambda x: x.date())
# master_df["YEAR"]=pd.DatetimeIndex(master_df["FROMDATE"]).year
# master_df["MONTH"]=pd.DatetimeIndex(master_df["FROMDATE"]).month
# master_df.head()
master_df["FROMDATE"]=pd.to_datetime(master_df['FROMDATE'],unit='ms')
master_df["YEAR"]=pd.DatetimeIndex(master_df["FROMDATE"]).year
master_df

In [ ]:
master_df["YEAR"].min()
# master_df.columns

In [ ]:
# Create dataframes for each ward containing all DC crash data# Create 
ward1DF = master_df.loc[master_df["WARD"]== "Ward 1"]
ward2DF = master_df.loc[master_df["WARD"]== "Ward 2"]
ward3DF = master_df.loc[master_df["WARD"]== "Ward 3"]
ward4DF = master_df.loc[master_df["WARD"]== "Ward 4"]
ward5DF = master_df.loc[master_df["WARD"]== "Ward 5"]
ward6DF = master_df.loc[master_df["WARD"]== "Ward 6"]
ward7DF = master_df.loc[master_df["WARD"]== "Ward 7"]
ward8DF = master_df.loc[master_df["WARD"]== "Ward 8"]

In [ ]:
#Create dataframes for each ward for bike crashes
ward1_bike_df = ward1DF.loc[ward1DF["TOTAL_BICYCLES"] >= 1]
ward2_bike_df = ward2DF.loc[ward2DF["TOTAL_BICYCLES"] >= 1]
ward3_bike_df = ward3DF.loc[ward3DF["TOTAL_BICYCLES"] >= 1]
ward4_bike_df = ward4DF.loc[ward4DF["TOTAL_BICYCLES"] >= 1]
ward5_bike_df = ward5DF.loc[ward5DF["TOTAL_BICYCLES"] >= 1]
ward6_bike_df = ward6DF.loc[ward6DF["TOTAL_BICYCLES"] >= 1]
ward7_bike_df = ward7DF.loc[ward7DF["TOTAL_BICYCLES"] >= 1]
ward8_bike_df = ward8DF.loc[ward8DF["TOTAL_BICYCLES"] >= 1]

#Create dataframes for each ward for car crashes
ward1_car_df = ward1DF.loc[ward1DF["TOTAL_VEHICLES"] >= 1]
ward2_car_df = ward2DF.loc[ward2DF["TOTAL_VEHICLES"] >= 1]
ward3_car_df = ward3DF.loc[ward3DF["TOTAL_VEHICLES"] >= 1]
ward4_car_df = ward4DF.loc[ward4DF["TOTAL_VEHICLES"] >= 1]
ward5_car_df = ward5DF.loc[ward5DF["TOTAL_VEHICLES"] >= 1]
ward6_car_df = ward6DF.loc[ward6DF["TOTAL_VEHICLES"] >= 1]
ward7_car_df = ward7DF.loc[ward7DF["TOTAL_VEHICLES"] >= 1]
ward8_car_df = ward8DF.loc[ward8DF["TOTAL_VEHICLES"] >= 1]

#Create dataframes for each ward for pedestrians
ward1_pedestrian_df = ward1DF[ward1DF["TOTAL_PEDESTRIANS"] >= 1]
ward2_pedestrian_df = ward2DF[ward2DF["TOTAL_PEDESTRIANS"] >= 1]
ward3_pedestrian_df = ward3DF[ward3DF["TOTAL_PEDESTRIANS"] >= 1]
ward4_pedestrian_df = ward4DF[ward4DF["TOTAL_PEDESTRIANS"] >= 1]
ward5_pedestrian_df = ward5DF[ward5DF["TOTAL_PEDESTRIANS"] >= 1]
ward6_pedestrian_df = ward6DF[ward6DF["TOTAL_PEDESTRIANS"] >= 1]
ward7_pedestrian_df = ward7DF[ward7DF["TOTAL_PEDESTRIANS"] >= 1]
ward8_pedestrian_df = ward8DF[ward8DF["TOTAL_PEDESTRIANS"] >= 1]

#Create dataframes for each ward for taxis
ward1_taxi_df = ward1DF[ward1DF["TOTAL_TAXIS"] >= 1]
ward2_taxi_df = ward2DF[ward2DF["TOTAL_TAXIS"] >= 1]
ward3_taxi_df = ward3DF[ward3DF["TOTAL_TAXIS"] >= 1]
ward4_taxi_df = ward4DF[ward4DF["TOTAL_TAXIS"] >= 1]
ward5_taxi_df = ward5DF[ward5DF["TOTAL_TAXIS"] >= 1]
ward6_taxi_df = ward6DF[ward6DF["TOTAL_TAXIS"] >= 1]
ward7_taxi_df = ward7DF[ward7DF["TOTAL_TAXIS"] >= 1]
ward8_taxi_df = ward8DF[ward8DF["TOTAL_TAXIS"] >= 1]

#Create dataframes for each ward for government
ward1_gov_df = ward1DF[ward1DF["TOTAL_GOVERNMENT"] >= 1]
ward2_gov_df = ward2DF[ward2DF["TOTAL_GOVERNMENT"] >= 1]
ward3_gov_df = ward3DF[ward3DF["TOTAL_GOVERNMENT"] >= 1]
ward4_gov_df = ward4DF[ward4DF["TOTAL_GOVERNMENT"] >= 1]
ward5_gov_df = ward5DF[ward5DF["TOTAL_GOVERNMENT"] >= 1]
ward6_gov_df = ward6DF[ward6DF["TOTAL_GOVERNMENT"] >= 1]
ward7_gov_df = ward7DF[ward7DF["TOTAL_GOVERNMENT"] >= 1]
ward8_gov_df = ward8DF[ward8DF["TOTAL_GOVERNMENT"] >= 1]


In [ ]:
#Get locations of accidents in each ward

ward1_bikeLocations = ward1_bike_df[['LATITUDE', 'LONGITUDE']]
ward2_bikeLocations = ward2_bike_df[['LATITUDE', 'LONGITUDE']]
ward3_bikeLocations = ward3_bike_df[['LATITUDE', 'LONGITUDE']]
ward4_bikeLocations = ward4_bike_df[['LATITUDE', 'LONGITUDE']]
ward5_bikeLocations = ward5_bike_df[['LATITUDE', 'LONGITUDE']]
ward6_bikeLocations = ward6_bike_df[['LATITUDE', 'LONGITUDE']]
ward7_bikeLocations = ward7_bike_df[['LATITUDE', 'LONGITUDE']]
ward8_bikeLocations = ward8_bike_df[['LATITUDE', 'LONGITUDE']]


ward1_carLocations= ward1_car_df[['LATITUDE', 'LONGITUDE']]
ward2_carLocations= ward2_car_df[['LATITUDE', 'LONGITUDE']]
ward3_carLocations= ward3_car_df[['LATITUDE', 'LONGITUDE']]
ward4_carLocations= ward4_car_df[['LATITUDE', 'LONGITUDE']]
ward5_carLocations= ward5_car_df[['LATITUDE', 'LONGITUDE']]
ward6_carLocations= ward6_car_df[['LATITUDE', 'LONGITUDE']]
ward7_carLocations= ward7_car_df[['LATITUDE', 'LONGITUDE']]
ward8_carLocations= ward8_car_df[['LATITUDE', 'LONGITUDE']]

ward1_pedestrianLocations = ward1_pedestrian_df[['LATITUDE', 'LONGITUDE']]
ward2_pedestrianLocations = ward2_pedestrian_df[['LATITUDE', 'LONGITUDE']]
ward3_pedestrianLocations = ward3_pedestrian_df[['LATITUDE', 'LONGITUDE']]
ward4_pedestrianLocations = ward4_pedestrian_df[['LATITUDE', 'LONGITUDE']]
ward5_pedestrianLocations = ward5_pedestrian_df[['LATITUDE', 'LONGITUDE']]
ward6_pedestrianLocations = ward6_pedestrian_df[['LATITUDE', 'LONGITUDE']]
ward7_pedestrianLocations = ward7_pedestrian_df[['LATITUDE', 'LONGITUDE']]
ward8_pedestrianLocations = ward8_pedestrian_df[['LATITUDE', 'LONGITUDE']]

ward1_taxiLocations = ward1_taxi_df[['LATITUDE', 'LONGITUDE']]
ward2_taxiLocations = ward2_taxi_df[['LATITUDE', 'LONGITUDE']]
ward3_taxiLocations = ward3_taxi_df[['LATITUDE', 'LONGITUDE']]
ward4_taxiLocations = ward4_taxi_df[['LATITUDE', 'LONGITUDE']]
ward5_taxiLocations = ward5_taxi_df[['LATITUDE', 'LONGITUDE']]
ward6_taxiLocations = ward6_taxi_df[['LATITUDE', 'LONGITUDE']]
ward7_taxiLocations = ward7_taxi_df[['LATITUDE', 'LONGITUDE']]
ward8_taxiLocations = ward8_taxi_df[['LATITUDE', 'LONGITUDE']]

ward1_govLocations = ward1_gov_df[['LATITUDE', 'LONGITUDE']]
ward2_govLocations = ward2_gov_df[['LATITUDE', 'LONGITUDE']]
ward3_govLocations = ward3_gov_df[['LATITUDE', 'LONGITUDE']]
ward4_govLocations = ward4_gov_df[['LATITUDE', 'LONGITUDE']]
ward5_govLocations = ward5_gov_df[['LATITUDE', 'LONGITUDE']]
ward6_govLocations = ward6_gov_df[['LATITUDE', 'LONGITUDE']]
ward7_govLocations = ward7_gov_df[['LATITUDE', 'LONGITUDE']]
ward8_govLocations = ward8_gov_df[['LATITUDE', 'LONGITUDE']]

In [ ]:
#Ward 1 Map
ward1_bikeLayer = gmaps.symbol_layer(ward1_bikeLocations, fill_color = 'red', stroke_color = 'red')
ward1_carLayer = gmaps.symbol_layer(ward1_carLocations, fill_color = 'blue', stroke_color = 'blue')
ward1_pedestrianLayer = gmaps.symbol_layer(ward1_pedestrianLocations, fill_color = 'purple', stroke_color = 'purple')
ward1_taxiLayer = gmaps.symbol_layer(ward1_taxiLocations, fill_color = 'green', stroke_color = 'green')
ward1_govLayer = gmaps.symbol_layer(ward1_govLocations, fill_color = 'yellow', stroke_color = 'yellow')

ward1Map = gmaps.figure()
ward1Map.add_layer(ward1_bikeLayer)
ward1Map.add_layer(ward1_carLayer)
ward1Map.add_layer(ward1_pedestrianLayer)
ward1Map.add_layer(ward1_taxiLayer)
ward1Map.add_layer(ward1_govLayer)
ward1Map

In [ ]:
#Ward 2 Map
ward2_bikeLayer = gmaps.symbol_layer(ward2_bikeLocations, fill_color = 'red', stroke_color = 'red')
ward2_carLayer = gmaps.symbol_layer(ward2_carLocations, fill_color = 'blue', stroke_color = 'blue')
ward2_pedestrianLayer = gmaps.symbol_layer(ward2_pedestrianLocations, fill_color = 'purple', stroke_color = 'purple')
ward2_taxiLayer = gmaps.symbol_layer(ward2_taxiLocations, fill_color = 'green', stroke_color = 'green')
ward2_govLayer = gmaps.symbol_layer(ward2_govLocations, fill_color = 'yellow', stroke_color = 'yellow')

ward2Map = gmaps.figure()
ward2Map.add_layer(ward2_bikeLayer)
ward2Map.add_layer(ward2_carLayer)
ward2Map.add_layer(ward2_pedestrianLayer)
ward2Map.add_layer(ward2_taxiLayer)
ward2Map.add_layer(ward2_govLayer)
ward2Map

In [ ]:
#Ward 3 Map
ward3_bikeLayer = gmaps.symbol_layer(ward3_bikeLocations, fill_color = 'red', stroke_color = 'red')
ward3_carLayer = gmaps.symbol_layer(ward3_carLocations, fill_color = 'blue', stroke_color = 'blue')
ward3_pedestrianLayer = gmaps.symbol_layer(ward3_pedestrianLocations, fill_color = 'purple', stroke_color = 'purple')
ward3_taxiLayer = gmaps.symbol_layer(ward3_taxiLocations, fill_color = 'green', stroke_color = 'green')
ward3_govLayer = gmaps.symbol_layer(ward3_govLocations, fill_color = 'yellow', stroke_color = 'yellow')

ward3Map = gmaps.figure()
ward3Map.add_layer(ward3_bikeLayer)
ward3Map.add_layer(ward3_carLayer)
ward3Map.add_layer(ward3_pedestrianLayer)
ward3Map.add_layer(ward3_taxiLayer)
ward3Map.add_layer(ward3_govLayer)
ward3Map

In [ ]:
#Ward 4 Map
ward4_bikeLayer = gmaps.symbol_layer(ward4_bikeLocations, fill_color = 'red', stroke_color = 'red')
ward4_carLayer = gmaps.symbol_layer(ward4_carLocations, fill_color = 'blue', stroke_color = 'blue')
ward4_pedestrianLayer = gmaps.symbol_layer(ward4_pedestrianLocations, fill_color = 'purple', stroke_color = 'purple')
ward4_taxiLayer = gmaps.symbol_layer(ward4_taxiLocations, fill_color = 'green', stroke_color = 'green')
ward4_govLayer = gmaps.symbol_layer(ward4_govLocations, fill_color = 'yellow', stroke_color = 'yellow')

ward4Map = gmaps.figure()
ward4Map.add_layer(ward4_bikeLayer)
ward4Map.add_layer(ward4_carLayer)
ward4Map.add_layer(ward4_pedestrianLayer)
ward4Map.add_layer(ward4_taxiLayer)
ward4Map.add_layer(ward4_govLayer)
ward4Map

In [ ]:
#Ward 5 Map
ward5_bikeLayer = gmaps.symbol_layer(ward5_bikeLocations, fill_color = 'red', stroke_color = 'red')
ward5_carLayer = gmaps.symbol_layer(ward5_carLocations, fill_color = 'blue', stroke_color = 'blue')
ward5_pedestrianLayer = gmaps.symbol_layer(ward5_pedestrianLocations, fill_color = 'purple', stroke_color = 'purple')
ward5_taxiLayer = gmaps.symbol_layer(ward5_taxiLocations, fill_color = 'green', stroke_color = 'green')
ward5_govLayer = gmaps.symbol_layer(ward5_govLocations, fill_color = 'yellow', stroke_color = 'yellow')

ward5Map = gmaps.figure()
ward5Map.add_layer(ward5_bikeLayer)
ward5Map.add_layer(ward5_carLayer)
ward5Map.add_layer(ward5_pedestrianLayer)
ward5Map.add_layer(ward5_taxiLayer)
ward5Map.add_layer(ward5_govLayer)
ward5Map

In [ ]:
#Ward 6 Map
ward6_bikeLayer = gmaps.symbol_layer(ward6_bikeLocations, fill_color = 'red', stroke_color = 'red')
ward6_carLayer = gmaps.symbol_layer(ward6_carLocations, fill_color = 'blue', stroke_color = 'blue')
ward6_pedestrianLayer = gmaps.symbol_layer(ward6_pedestrianLocations, fill_color = 'purple', stroke_color = 'purple')
ward6_taxiLayer = gmaps.symbol_layer(ward6_taxiLocations, fill_color = 'green', stroke_color = 'green')
ward6_govLayer = gmaps.symbol_layer(ward6_govLocations, fill_color = 'yellow', stroke_color = 'yellow')

ward6Map = gmaps.figure()
ward6Map.add_layer(ward6_bikeLayer)
ward6Map.add_layer(ward6_carLayer)
ward6Map.add_layer(ward6_pedestrianLayer)
ward6Map.add_layer(ward6_taxiLayer)
ward6Map.add_layer(ward6_govLayer)
ward6Map

In [ ]:
#Ward 7 Map
ward7_bikeLayer = gmaps.symbol_layer(ward7_bikeLocations, fill_color = 'red', stroke_color = 'red')
ward7_carLayer = gmaps.symbol_layer(ward7_carLocations, fill_color = 'blue', stroke_color = 'blue')
ward7_pedestrianLayer = gmaps.symbol_layer(ward7_pedestrianLocations, fill_color = 'purple', stroke_color = 'purple')
ward7_taxiLayer = gmaps.symbol_layer(ward7_taxiLocations, fill_color = 'green', stroke_color = 'green')
ward7_govLayer = gmaps.symbol_layer(ward7_govLocations, fill_color = 'yellow', stroke_color = 'yellow')

ward7Map = gmaps.figure()
ward7Map.add_layer(ward7_bikeLayer)
ward7Map.add_layer(ward7_carLayer)
ward7Map.add_layer(ward7_pedestrianLayer)
ward7Map.add_layer(ward7_taxiLayer)
ward7Map.add_layer(ward7_govLayer)
ward7Map

In [ ]:
#Ward 8 Map
ward8_bikeLayer = gmaps.symbol_layer(ward8_bikeLocations, fill_color = 'red', stroke_color = 'red')
ward8_carLayer = gmaps.symbol_layer(ward8_carLocations, fill_color = 'blue', stroke_color = 'blue')
ward8_pedestrianLayer = gmaps.symbol_layer(ward8_pedestrianLocations, fill_color = 'purple', stroke_color = 'purple')
ward8_taxiLayer = gmaps.symbol_layer(ward8_taxiLocations, fill_color = 'green', stroke_color = 'green')
ward8_govLayer = gmaps.symbol_layer(ward8_govLocations, fill_color = 'yellow', stroke_color = 'yellow')

ward8Map = gmaps.figure()
ward8Map.add_layer(ward8_bikeLayer)
ward8Map.add_layer(ward8_carLayer)
ward8Map.add_layer(ward8_pedestrianLayer)
ward8Map.add_layer(ward8_taxiLayer)
ward8Map.add_layer(ward8_govLayer)
ward8Map

In [ ]:
#Create dataframes for each accident type
bike_df = master_df.loc[master_df["TOTAL_BICYCLES"] >= 1]
driver_df = master_df.loc[master_df["TOTAL_VEHICLES"] >= 1]
pedestrian_df = master_df.loc[master_df["TOTAL_PEDESTRIANS"] >= 1]
taxi_df = master_df.loc[master_df["TOTAL_TAXIS"] >= 1]
gov_df = master_df.loc[master_df["TOTAL_GOVERMENT"] >= 1]

In [ ]:
bike_count = len(bike_df)
driver_count = len(driver_df)
pedestrian_count = len(pedestrian_df)
taxi_count = len(taxi_df)
gov_count = len(gov_df)

In [ ]:
types = ["Bike", "Car", "Pedestrian", "Taxi", "Government"]
count = [bike_count, driver_count, pedestrian_count, taxi_count, gov_count]
colors = ["red", "blue", "purple", "green", "yellow"]
explode = (.3 ,.5 ,.3, 0, 0)

total_accidents = plt.pie(count, labels = types, colors = colors, autopct="%1.1f%%", shadow = True, explode = explode)
plt.axis("equal")

In [ ]:
# bike_locations = bike_df[['LATITUDE', 'LONGITUDE']]
# driver_locations = driver_df[['LATITUDE', 'LONGITUDE']]
# pedestrian_locations = pedestrian_df[['LATITUDE', 'LONGITUDE']]

In [ ]:
# bike_layer = gmaps.symbol_layer(bike_locations, fill_color = 'blue', stroke_color = 'blue')
# driver_layer = gmaps.symbol_layer(driver_locations, fill_color = 'red', stroke_color = 'red')
# pedestrian_layer = gmaps.symbol_layer(pedestrian_locations, fill_color = 'yellow', stroke_color = 'yellow')

In [ ]:
# fig = gmaps.figure()
# fig.add_layer(bike_layer)
# fig.add_layer(driver_layer)
# fig.add_layer(pedestrian_layer)
# fig

In [ ]:
# bike_df_reduced = bike_df[["ADDRESS", "CRIMEID", "FATAL_BICYCLIST", "FROMDATE", "LATITUDE", "LONGITUDE", "MAJORINJURIES_BICYCLIST", "MINORINJURIES_BICYCLIST","TOTAL_BICYCLES", "XCOORD", "YCOORD"]]
# bike_df_reduced

In [ ]:
# marker_locations = bike_df_reduced[["LATITUDE", "LONGITUDE"]]